In [ ]:
#import library
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import IPython.display as ipd
import librosa
import librosa.display
from scipy.io import wavfile as wav
from scipy import signal
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from sklearn import metrics
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

%matplotlib inline

**Batas**

In [ ]:
wav_file = "../input/data-burung/essential_data/src_wavs/PC10_20090513_054500_0010.wav"

In [ ]:
plt.figure(figsize=(14,5))
data, sample_rate = librosa.load(wav_file)
librosa.display.waveplot(data, sr=sample_rate)
ipd.Audio(wav_file)

In [ ]:
n_fft=2048
hop_length=512
# Short-time Fourier transform 
D_astfly = np.abs(librosa.stft(data, n_fft = n_fft, hop_length = hop_length))
# Merubah amplitudo spectrogram menjadi Decibels-scaled spectrogram.
DB_astfly = librosa.amplitude_to_db(D_astfly, ref = np.max)

fig = plt.Figure()
canvas = FigureCanvas(fig)
ax = fig.add_subplot(111)
img=librosa.display.specshow(DB_astfly, sr = sample_rate, hop_length = hop_length, x_axis = 'time', y_axis = 'log', ax=ax)
fig.savefig('spectrogram.png')

In [ ]:
sample_rate

In [ ]:
data

In [ ]:
species = pd.read_csv('../input/data-burung/essential_data/species_list.txt')
species

In [ ]:
file_name = pd.read_csv('../input/data-burung/essential_data/rec_id2filename.txt')
file_name

In [ ]:
id_ls = []
label_ls = []

num_species = 19
raw =  pd.read_csv('../input/data-burung/essential_data/rec_labels_test_hidden.txt', sep = ';')
label = np.zeros([len(raw),num_species])
for i in range(len(raw)):
    line = raw.iloc[i]
    labels = line[0].split(',')
    id_ls.append(labels.pop(0))
    if len(labels) != 0:
        label_ls.append(labels[0])
    else:
#         label_ls.append(np.nan)
        label_ls.append(-1)

label_df = pd.DataFrame(list(zip(id_ls, label_ls)), columns =['rec_id', 'label'])
label_df

In [ ]:
folds = pd.read_csv('../input/data-burung/essential_data/CVfolds_2.txt')
folds

In [ ]:
bird_df = pd.DataFrame()
bird_df['rec_id'] = file_name['rec_id'].values
bird_df['filename'] = file_name['filename'].values
bird_df['label'] = label_df['label']
bird_df['fold'] = folds['fold'].values

In [ ]:
bird_df.loc[bird_df['fold'] == 1]

In [ ]:
data_ls = []
samplenr_ls = []
# sample_ls = []
for name in file_name["filename"]:
    data, samplenr_rate = librosa.load(r'../input/data-burung/essential_data/src_wavs/'+str(name)+'.wav')
#     sample_rate, samples = wav.read(r'../input/data-burung/essential_data/src_wavs/'+str(name)+'.wav')
    data_ls.append(data)
    samplenr_ls.append(samplenr_rate)
#     sample_ls.append(sample_rate)

In [ ]:
# wav_df = pd.DataFrame()
# bird_df['rec_id'] = file_name["rec_id"].values
# bird_df['filename'] = file_name["filename"].values
bird_df['feature'] = data_ls
bird_df['norm_sample_rate'] = samplenr_ls
# bird_df['sample_rate'] = sample_ls
# wav_df = wav_df.reset_index(drop=True)
bird_df

In [ ]:
bird_df.shape

In [ ]:
bird_train_df = bird_df.loc[bird_df['fold'] == 0]
bird_train_df = bird_train_df.reset_index(drop=True)
bird_test_df = bird_df.loc[bird_df['fold'] == 1]
bird_test_df = bird_test_df.reset_index(drop=True)
# bird_test_df['label'] = np.nan

In [ ]:
bird_train_df

In [ ]:
bird_test_df

In [ ]:
bird_df['label'].value_counts(normalize=True)

In [ ]:
def extract_features(filename):
    
    # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('voice')+'/'+str(filename))

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
#     data, samplenr_rate = librosa.load(r'../input/data-burung/essential_data/src_wavs/'+str(name)+'.wav')
    X, sample_rate = librosa.load(r'../input/data-burung/essential_data/src_wavs/'+str(filename)+'.wav', res_type='kaiser_fast')
#     X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    label = filename

    return mfccs, chroma, mel, contrast, tonnetz, label

In [ ]:
features_label = bird_train_df['filename'].apply(extract_features)
features_test = bird_test_df['filename'].apply(extract_features)

In [ ]:
features_test

In [ ]:
features = []
for i in range(len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))

features_test_ls = []
for i in range(len(features_test)):
    features_test_ls.append(np.concatenate((features_test[i][0], features_test[i][1], 
                features_test[i][2], features_test[i][3],
                features_test[i][4]), axis=0))

In [ ]:
len(features)

In [ ]:
X = np.array(features)
X_test = np.array(features_test_ls)
y = np.array(bird_train_df['label'].astype('int'))

In [ ]:
lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))

In [ ]:
X_test.shape

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train = X[:257]
y_train = y[:257]

X_val = X[257:]
y_val = y[257:]

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [ ]:
model = Sequential()

model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(20, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [ ]:
history = model.fit(X_train, y_train, batch_size=5, epochs=10,validation_data=(X_val, y_val), callbacks=[early_stop])

In [ ]:
preds = model.predict(X_test)
preds

In [ ]:
label_pred = np.argmax(preds, axis=1)
label_pred

In [ ]:
len(label_pred)

In [ ]:
bird_test_df['label'] = label_pred
bird_test_df

In [ ]:
submission = pd.read_csv('../input/data-burung/essential_data/sample_submission.csv')
submission

In [ ]:
submission.loc[(submission['rec_id'] == 0) & (submission['species'] == 0), 'probability']

In [ ]:
for i, row in bird_test_df.iterrows():
    submission.loc[(submission['rec_id'] == row['rec_id']) & (submission['species'] == row['label']), 'probability'] = 1
#     submission[(submission['rec_id'] == row['rec_id']) & (submission['species'] == row['label'])]['probability'] = 1

submission

In [ ]:
submission[(submission['probability'] == 1)]

In [ ]:
# submission.loc[submission['species'] == 1, 'probability'] = 1
submission.insert(0, "id", (submission["rec_id"] * 100) + submission["species"])
submission = submission.drop(['rec_id', 'species'], axis=1)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)